# Face Mask Detection - Image Processing and Computer Vision Project

## By Sri Ram Sai Pavan Relangi

## Importing Libraries

In [16]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Preparing the dataset

In [17]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (128, 128))
            images.append(img)
    return images

In [18]:
with_mask_images = load_images_from_folder("Face_Mask_Dataset/Mask")
without_mask_images = load_images_from_folder("Face_Mask_Dataset/No Mask")

### Converting Images to numpy arrays

In [19]:
with_mask_images = np.array(with_mask_images)
without_mask_images = np.array(without_mask_images)

### Creating corresponding labels

In [20]:
with_mask_labels = np.ones(len(with_mask_images))
without_mask_labels = np.zeros(len(without_mask_images))

### Splitting the dataset into training and testing sets

In [21]:
X_train = np.concatenate((with_mask_images[:8000], without_mask_images[:8000]), axis=0)
y_train = np.concatenate((with_mask_labels[:8000], without_mask_labels[:8000]), axis=0)
X_test = np.concatenate((with_mask_images[8000:], without_mask_images[8000:]), axis=0)
y_test = np.concatenate((with_mask_labels[8000:], without_mask_labels[8000:]), axis=0)

### Normalize the Image data

In [22]:
X_train = X_train / 255.0
X_test = X_test / 255.0

## Building CNN Model (Convolutional Neural Networks)

In [23]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Training the model

In [25]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
11/11 [==============================] - 9s 530ms/step - loss: 1.1014 - accuracy: 0.5490
Epoch 2/10
11/11 [==============================] - 5s 481ms/step - loss: 0.5700 - accuracy: 0.7359
Epoch 3/10
11/11 [==============================] - 5s 440ms/step - loss: 0.4250 - accuracy: 0.8487
Epoch 4/10
11/11 [==============================] - 5s 430ms/step - loss: 0.3567 - accuracy: 0.8576
Epoch 5/10
11/11 [==============================] - 5s 457ms/step - loss: 0.2502 - accuracy: 0.9080
Epoch 6/10
11/11 [==============================] - 5s 486ms/step - loss: 0.1955 - accuracy: 0.9169
Epoch 7/10
11/11 [==============================] - 5s 450ms/step - loss: 0.1492 - accuracy: 0.9674
Epoch 8/10
11/11 [==============================] - 6s 525ms/step - loss: 0.1190 - accuracy: 0.9614
Epoch 9/10
11/11 [==============================] - 5s 492ms/step - loss: 0.0720 - accuracy: 0.9822
Epoch 10/10
11/11 [==============================] - 5s 472ms/step - loss: 0.0478 - accuracy: 0.9881

## Saving the model

In [26]:
model.save('mask_detection_model.h5')

## Testing the Model

In [28]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_roi = frame[y:y + h, x:x + w]
        face_roi = cv2.resize(face_roi, (128, 128))
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = face_roi / 255.0

        prediction = model.predict(face_roi)
        if prediction > 0.5:
            label = 'With Mask'
        else:
            label = 'Without Mask'

        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 24ms/step
